# Rechunk the kerchunked dataset

In [ ]:
import fsspec
import fsspec.implementations.reference
import zarr
import xarray as xr
from pathlib import Path

from rechunker import rechunk

In [ ]:
import rechunker
rechunker.__version__

In [ ]:
import zarr
zarr.__version__

#### Start a Dask Gateway cluster
Use a custom helper function `ebd.start_dask_cluster` to set options on this cluster.  We don't have to use this helper, it just cuts down on lines of code in notebooks.

In [ ]:
import sys
import os
sys.path.append('/shared/users/rsignell/lib')
import ebdpy as ebd
os.environ['AWS_PROFILE'] = 'esip-qhub'  # use env vars for AWS credentials to write

client, cluster, gateway = ebd.start_dask_cluster(
    profile=os.environ['AWS_PROFILE'],
    worker_max=30,
    region='us-west-2', 
    worker_profile='Medium Worker',
    use_existing_cluster=False,
    adaptive_scaling=False, 
    wait_for_cluster=False, 
    propagate_env=True)

#### Open Kerchunked Dataset JSON for one year

In [ ]:
#client.close()

#### Open Entire Kerchunked Dataset, lazy loaded from parquet

In [ ]:
s3_lazy_refs = 's3://esip-qhub-public/nwm/LDAS-1k/lazyrefs'

In [ ]:
%%time
fs = fsspec.implementations.reference.DFReferenceFileSystem(s3_lazy_refs, lazy=True, target_options={"anon": True},
                                                            remote_protocol="s3", remote_options={"anon": True})
m = fs.get_mapper("")
ds = xr.open_dataset(m, engine="zarr", chunks={'time':1, 'y':3840, 'x':4608}, 
                     backend_kwargs=dict(consolidated=False))

In [ ]:
ds

#### Select high-priority vars only

In [ ]:
ds = ds[['ACCET', 'SNEQV', 'FSNO', 'crs']]

In [ ]:
ds

In [ ]:
ds['ACCET'].isel(time=slice(0,144))

In [ ]:
ds.attrs

#### set up zarr stores for temporary and final zarr stores on S3

In [ ]:
fs_write = fsspec.filesystem('s3', anon=False, skip_instance_cache=True)

In [ ]:
temp_name = 'esip-qhub/testing/usgs/nwm1km.tmp'
target_name = 'esip-qhub/testing/usgs/nwm1km.zarr'

In [ ]:
fs_write.rm(temp_name, recursive=True)

In [ ]:
fs_write.rm(target_name, recursive=True)

In [ ]:
temp_store = fs_write.get_mapper(temp_name)
target_store = fs_write.get_mapper(target_name)

In [ ]:
ds = ds.drop('crs')

In [ ]:
a = len(ds.time)/(144/2)

In [ ]:
b = (len(ds.x) * len(ds.y))/((96*2)*(132*2))

In [ ]:
a/b

In [ ]:
#client.close()

In [ ]:
#from dask.distributed import Client

In [ ]:
#client = Client(threads_per_worker=1)

In [ ]:
#client.amm.start()

#### Rechunk!

In [ ]:
rechunked = rechunk(ds.isel(time=slice(0,144)), target_chunks={'y':96*2, 'x':132*2, 'time':144/2},
                    target_store=target_store, temp_store=temp_store, max_mem='2.8GiB')

In [ ]:
%%time
rechunked.execute(retries=10)

In [ ]:
zarr.convenience.consolidate_metadata(target_store)

#### Explore the rechunked dataset

In [ ]:
ds2 = xr.open_dataset(target_store, engine='zarr', chunks={})

In [ ]:
ds2

In [ ]:
ds2.ACCET

In [ ]:
import hvplot.xarray


In [ ]:
ds2.ACCET[:,2000,2000].hvplot(x='time')

In [ ]:
client.close()